In [1]:
# start julia in terminal with export JULIA_NUM_THREADS=5
# change path e.g. "cd("./Documents/Github/Multi_NNGP/RDA/projects/Conj/")"
# run code line by line

# This file is for illustration, the RDA results are presented in the paper

In [ ]:
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using DataFrames
using Dates

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
@load "../../data/conj_data/conj_data_expanded.jld"

┌ Warning: type OrderedCollections.OrderedDict{Symbol,Any} does not exist in workspace; reconstructing
└ @ JLD2 /home/lu/.julia/packages/JLD2/KjBIK/src/data.jl:1153


24-element Array{Symbol,1}:
 :U_indx      
 :S_indx      
 :N           
 :n           
 :m           
 :NN          
 :nnIndx_col  
 :nnIndx_row  
 :NM          
 :MnnIndxLU   
 :MnnIndx     
 :Mnndists    
 :coords      
 :coords_pred 
 :CVNN_train  
 :CVnnIndx_col
 :CVnnIndx_row
 :CVNN_test   
 :CVMnnIndxLU 
 :CVMnnIndx   
 :CVMnndists  
 :parts_train 
 :parts_test  
 :kf          

In [4]:
using RData
@time data_cleaned = load("../../data/rawdata/cleaned_data2_expanded.RData", convert = true)["data_cleaned2"];
@time hold_index_set = load("../../data/rawdata/hold_index_expanded.RData", convert = true);

  9.158490 seconds (26.33 M allocations: 2.557 GiB, 16.42% gc time)
  0.169142 seconds (49.53 k allocations: 15.956 MiB, 8.37% gc time)


In [5]:
DataFrames.describe(data_cleaned)

,variable,mean,min,median,max
,Symbol,Float64,Float64,Float64,Float64
1,x,-1.04265e7,-1.11193e7,-1.03937e7,-1.00078e7
2,y,3.83719e6,3.33608e6,3.81747e6,4.44757e6
3,scaled_x,0.692742,0.0,0.725548,1.11149
4,scaled_y,0.501102,0.0,0.481382,1.11149
5,NDVI,0.240135,-0.00954541,0.192437,0.666598
6,EVI,0.15018,-0.00410843,0.122129,0.618208
7,red_reflectance,0.144423,0.0031,0.1452,0.3997
8,NIR_reflectance,0.243497,0.005,0.2383,0.6276
9,blue_reflectance,0.0752172,0.0001,0.0754,0.2


In [22]:
Y_ord = convert(Matrix, data_cleaned[:, [:NDVI, :red_reflectance]]);
X_ord = dropzeros(sparse([fill(1.0, (N, 1)) convert(Matrix, data_cleaned[:, 28:28])]));#[3, 4, 28]
p = 2; q = 2; K = 2;

In [7]:
data_cleaned = nothing; hold_index_set = nothing; GC.gc();

In [8]:
Sys.free_memory()/(2^20*1024)

1.3651237487792969

In [8]:
# priors #
μβ = fill(0.0, p, q); inv_Vr =fill(0.0, p, p); # set Vr^{-1} be zero for the simulation...
νΣ = q + 1; ΨΣ = [[1.0 0.0]; [0.0 1.0]];

In [16]:
# (a) grid of candidate value
ϕ_grid = LinRange(3 / sqrt(2), 3 * 100 / sqrt(2), 10); #LinRange(3 / sqrt(2), 3 * 100 / sqrt(2), 10)
α_grid = LinRange(0.6, 0.999, 20) #LinRange(0.6, 0.999, 10)

20-element LinRange{Float64}:
 0.6,0.621,0.642,0.663,0.684,0.705,0.726,…,0.894,0.915,0.936,0.957,0.978,0.999

In [1]:
using Base.Threads
println("Number of threads = $(nthreads())")

Number of threads = 1


In [17]:
# For loop
CVRMSPE_i = fill(0.0, length(ϕ_grid), length(α_grid), kf);
prog = Progress(kf, 1, "Computing initial pass...", 50)

@time Threads.@threads for i in 1:kf
        print(Threads.threadid());
        CVnIndx = length(CVNN_train[i].nnIndx);
        nk = length(parts_test[i]);
        CVA = Array{Float64}(undef, CVnIndx); CVD = Array{Float64}(undef, n - nk);
        CVDI_AX = []; CVDI_AY = []; CVVstar = []; CVμstar = [];
    
        CVAtilde = Array{Float64}(undef, nk * m); CVDtilde = Array{Float64}(undef, nk);
        CVAtildeM = [];
    
@time        for j in 1:length(ϕ_grid) 
            for k in 1:length(α_grid)
            
                # obtain posterior estimators of beta
                getAD_collapse(coords[parts_train[i], :], 
                    CVNN_train[i].nnIndx, CVNN_train[i].nnDist, CVNN_train[i].nnIndxLU, 
                    ϕ_grid[j], 0.5, α_grid[k], CVA, CVD);
                CVDI_AX = Diagonal(1 ./ sqrt.(CVD)) * (sparse(CVnnIndx_row[i], CVnnIndx_col[i], 
                        vcat(-CVA, ones(n - nk))) * X_ord[S_indx[parts_train[i]], :]);
                CVDI_AY = Diagonal(1 ./ sqrt.(CVD)) * (sparse(CVnnIndx_row[i], CVnnIndx_col[i], 
                        vcat(-CVA, ones(n - nk))) * Y_ord[S_indx[parts_train[i]], :]);
                CVVstar = inv(Symmetric(CVDI_AX'CVDI_AX + inv_Vr));
                CVμstar = CVVstar * (CVDI_AX'CVDI_AY + inv_Vr * μβ);
            
                # obtain posterior predictions over holded folder
                getAD_collapse(coords[parts_train[i], :], CVMnnIndx[i], 
                    CVMnndists[i], CVMnnIndxLU[i],  ϕ_grid[j], 0.5, α_grid[k], CVAtilde, CVDtilde)
                CVAtildeM = sparse(repeat(1:nk, inner = m), CVMnnIndx[i], CVAtilde, nk, n - nk);
                Y_hold_mean = X_ord[S_indx[parts_test[i]], :] * CVμstar + 
                    CVAtildeM * (Y_ord[S_indx[parts_train[i]], :] - X_ord[S_indx[parts_train[i]], :] * CVμstar);
            
                # Calculate RMSPE
                CVRMSPE_i[j, k, i] = CVRMSPE_i[j, k, i] + mean((Y_ord[S_indx[parts_test[i]], :] - Y_hold_mean).^2) * 
                    (length(parts_test[i]) / n);
            end
        end
    end

1 45.792807 seconds (249.80 M allocations: 81.586 GiB, 16.51% gc time)
1 43.609500 seconds (241.46 M allocations: 81.178 GiB, 16.92% gc time)
1 43.752239 seconds (241.46 M allocations: 81.178 GiB, 17.00% gc time)
1 43.470574 seconds (241.46 M allocations: 81.178 GiB, 17.01% gc time)
1 43.683839 seconds (241.46 M allocations: 81.178 GiB, 17.01% gc time)


31542

11438.666772 seconds (51.14 G allocations: 16.313 TiB, 50.16% gc time)

11565.119542 seconds (51.73 G allocations: 16.498 TiB, 50.17% gc time)

11846.728617 seconds (52.98 G allocations: 16.904 TiB, 50.11% gc time)

11928.452885 seconds (53.32 G allocations: 17.013 TiB, 50.05% gc time)

12054.588710 seconds (53.70 G allocations: 17.139 TiB, 49.84% gc time)

12055.654125 seconds (53.70 G allocations: 17.140 TiB, 49.84% gc time)


In [16]:
#Dates.format(now(), "HH:MM:SS")  

In [18]:
CVRMSPE = [sqrt(sum(CVRMSPE_i[i, j, :])) for i in 1:length(ϕ_grid), j in 1:length(α_grid)]

2×2 Array{Float64,2}:
 0.0243681  0.0211972
 0.0229807  0.0208282

In [19]:
# cross-validation for choosing delta and phi ...
pick_ind = argmin(CVRMSPE);
ϕ_pick = ϕ_grid[pick_ind[1]]; α_pick = α_grid[pick_ind[2]]; # alpha ranges from 0 to 1. alpha = sigma2 / (sigma2 + tau2)
[ϕ_pick, α_pick]

2-element Array{Float64,1}:
 212.13203435596424
   0.999           

In [15]:
ϕ_pick = 17.918728638037667; α_pick = 0.999551052631579; #results from terminal...

In [20]:
# Build μ* V* Ψ* v*
nIndx = length(NN.nnIndx);
A = Array{Float64}(undef, nIndx); D = Array{Float64}(undef, n);
getAD_collapse(coords, NN.nnIndx, NN.nnDist, NN.nnIndxLU, ϕ_pick, 0.5, α_pick, A, D);
DI_AX = Diagonal(1 ./ sqrt.(D)) * (sparse(nnIndx_row, nnIndx_col, vcat(-A, ones(n))) * X_ord[S_indx, :]);
DI_AY = Diagonal(1 ./ sqrt.(D)) * (sparse(nnIndx_row, nnIndx_col, vcat(-A, ones(n))) * Y_ord[S_indx, :]);
Vstar = inv(Symmetric(DI_AX'DI_AX + inv_Vr));
μstar = Vstar * (DI_AX'DI_AY + inv_Vr * μβ);
Lμstar = cholesky(Vstar).L \ μstar;
Ψstar = ΨΣ + DI_AY'DI_AY + transpose(μβ) * inv_Vr * μβ - Lμstar'Lμstar;
νstar = νΣ + n;
Lstar = cholesky(Vstar);

In [18]:
# construct Atilde Dtilde #
Atilde = Array{Float64}(undef, NM * m); Dtilde = Array{Float64}(undef, NM);
getAD_collapse(coords, MnnIndx, Mnndists, MnnIndxLU, 
    ϕ_pick, 0.5, α_pick, Atilde, Dtilde)
AtildeM = sparse(repeat(1:NM, inner = m), MnnIndx, Atilde, NM, n);

In [13]:
# Preallocation for posterior samples #
N_sam = 500;
β_sam = Array{Float64, 3}(undef, p, q, N_sam);
Σ_sam = Array{Float64, 3}(undef, q, q, N_sam);
Y_m_sam = Array{Float64, 3}(undef, NM, q, N_sam);   # missing value

cholΣ_sam = Array{Float64, 2}(undef, q, q);

In [25]:
# generate the posterior samples #
Random.seed!(123);
prog = Progress(N_sam, 1, "Computing initial pass...", 50)
for l in 1:N_sam
    Σ_sam[:, :, l] = rand(InverseWishart(νstar, Ψstar), 1)[1];    # sample Σ
    cholΣ_sam = cholesky(Σ_sam[:, :, l]);
    β_sam[:, :, l] = (Lstar.L * rand(Normal(), p, q)) * 
                    cholΣ_sam.U + μstar;          # sample γ  
    Y_m_sam[:, :, l] = X_ord[U_indx, :] * β_sam[:, :, l] + 
    AtildeM * (Y_ord[S_indx, :] - X_ord[S_indx, :]* β_sam[:, :, l] ) + 
    Diagonal(sqrt.(Dtilde)) * rand(Normal(), NM, q) * cholΣ_sam.U
    next!(prog) # monitor the progress
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 0:00:13


| Time: 0:01:48

In [27]:
N_Inf_burn = 1;
Y_m_pos_qt = Array{Float64, 3}(undef, NM, q, 3);
Y_m_pos_mean = Array{Float64, 2}(undef, NM, q);
Y_m_pos_var = Array{Float64, 2}(undef, NM, q);
for j in 1:q
    for i in 1:NM
        Y_m_pos_qt[i, j, :] = quantile(Y_m_sam[i, j, N_Inf_burn:N_sam], [0.025, 0.5, 0.975]);
        Y_m_pos_mean[i, j] = mean(Y_m_sam[i, j, N_Inf_burn:N_sam]);
        Y_m_pos_var[i, j] = var(Y_m_sam[i, j, N_Inf_burn:N_sam]);
    end
end

In [1]:
@save "../../results/conj/conj_resp_results.jld" N_sam NM p q β_sam Σ_sam Y_m_pos_qt Y_m_pos_mean Y_m_pos_var U_indx Y_ord α_pick ϕ_pick

LoadError: UndefVarError: @save not defined